Alycia's attempt at gradient boosting model

In [15]:
import numpy as np

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import pandas as pd
#import sys, os
#sys.path.append(os.path.abspath('..'))

In [2]:
def report(results, n_top=10):
    """
    Generates formatted summary of gridsearch results. Mostly 
    adapted from scikit-learn documentation on gridsearch.
    """
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.5f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [3]:
x = np.load("data/x.npy")
y = np.load("data/y.npy")
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

gbm = GradientBoostingClassifier()

param_grid = {
    'learning_rate': [0.01, 0.5, 0.1],
    'max_depth': [2, 5, 10],
    'subsample': [0.8, 0.9],
    'max_features': [20, 50, 100],
}

clf = GridSearchCV(gbm, scoring='accuracy', param_grid=param_grid, 
                   n_jobs=4, verbose=2)
clf.fit(x_train, y_train)

report(clf.cv_results_)

print('Best Error: %f' %(clf.best_score_))
print('Best Model: %s' %(clf.best_params_))

C:\Users\alyci\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 54 candidates, totalling 162 fits


[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  2.9min
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed: 53.0min
[Parallel(n_jobs=4)]: Done 162 out of 162 | elapsed: 68.0min finished


Model with rank: 1
Mean validation score: 0.78258 (std: 0.00059)
Parameters: {'learning_rate': 0.1, 'max_depth': 5, 'max_features': 50, 'subsample': 0.9}

Model with rank: 2
Mean validation score: 0.78249 (std: 0.00123)
Parameters: {'learning_rate': 0.1, 'max_depth': 5, 'max_features': 100, 'subsample': 0.8}

Model with rank: 3
Mean validation score: 0.78142 (std: 0.00082)
Parameters: {'learning_rate': 0.1, 'max_depth': 5, 'max_features': 50, 'subsample': 0.8}

Model with rank: 4
Mean validation score: 0.78140 (std: 0.00116)
Parameters: {'learning_rate': 0.1, 'max_depth': 5, 'max_features': 20, 'subsample': 0.9}

Model with rank: 5
Mean validation score: 0.78062 (std: 0.00122)
Parameters: {'learning_rate': 0.1, 'max_depth': 5, 'max_features': 100, 'subsample': 0.9}

Model with rank: 6
Mean validation score: 0.78034 (std: 0.00102)
Parameters: {'learning_rate': 0.1, 'max_depth': 5, 'max_features': 20, 'subsample': 0.8}

Model with rank: 7
Mean validation score: 0.77956 (std: 0.00198)
Par

In [4]:
def generate_csv(model, x_data, name):
    ids = [i for i, _ in enumerate(x_data)]
    ids = np.array(ids)
    
    predictions = model.predict(x_data).flatten()
    
    df = pd.DataFrame()
    df['id'] = ids
    df['target'] = predictions
    df.to_csv(name + ".csv", index=False)

In [26]:
clf = GradientBoostingClassifier(learning_rate=0.1, max_depth=5, 
                                 subsample=0.9, max_features=300)

In [27]:
clf.fit(x_train, y_train)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=5,
              max_features=300, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=0.9, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [12]:
predictions = clf.predict(x_test)

In [28]:
print(clf.score(x_train, y_train))

0.8104602317315238


In [29]:
print(clf.score(x_test, y_test))

ValueError: Found input variables with inconsistent numbers of samples: [21341, 16000]

In [20]:
x_test = np.load("data/x_test.npy")

In [23]:
generate_csv(clf, x_test, "submissions/gbclassifier1")